In [0]:
from yaml import safe_load
from pyspark.sql.functions import col, lit, concat

with open('../params.yml', 'r') as f:
    params = safe_load(f)

CATALOG = params.get('data_params')['catalog']
SCHEMA = params.get('data_params')['schema']

df = (spark.read.option("readChangeData", "true")
      .option('startingVersion', '5')
      .table(f"{CATALOG}.{SCHEMA}.device_status_triage"))

df = df.withColumn('query_column', concat(col('device'), lit(' message: '), col('telemetry_error') ))

display(df)


In [0]:
def query_agent(device_error):
    prompt = make_prompt(device_error)
    response = client.predict(
        endpoint="ka-fce5e7c5-endpoint",
        inputs={
            "messages": [{"role": "user", "content": prompt}],
            "databricks_options": {"return_trace": True}
        }
    )
    return response.get('messages', [{'content': 'No response'}])[0]['content']

errorslist = df.select('query_column').toPandas().values.flatten().tolist()

def make_prompt(device_error):
    return f"""Please find an automated and remote fix for this device and error message: {device_error}. Be as concise as possible, preferably using fewer than 20 words. If such a solution exists without requiring human intervention, start the response with the phrase AUTOMATED FIX followed by a 1-sentence overview. If human intervention is necessary, start the response with the phrase HUMAN INTERVENTION REQUIRED and state a single phrase overview for the course of action needed.
    """
    
errorslist_solution = [make_prompt(x) for x in errorslist]

In [0]:
from mlflow.deployments import get_deploy_client

client = get_deploy_client("databricks")
response = client.predict(
    endpoint="agents_telecommunications-self_healing_networks-vendor_manuals",
    inputs={
        "messages": [{"role": "user", "content": x} for x in errorslist_solution],
        "temperature": 0.2,
        "n": 1,
        "max_tokens": 500
        }
    
)
print(response)

In [0]:
responselist = [x for x in response['messages'][-1]['content'].split('\n') if x != ''][1:]
devicelist = [responselist[i] for i in range(len(responselist)) if i % 2 == 0]
responselist = [responselist[i] for i in range(len(responselist)) if i % 2 == 1]
print(responselist)
# df = df.withColumn('solution', lit(responselist))]

In [0]:
from collections import OrderedDict
import pandas as pd
from pyspark.sql.functions import current_timestamp

errorslist_dedupe = list(OrderedDict.fromkeys(errorslist))
solutions_df = pd.DataFrame({'error_message': errorslist_dedupe, 'device_info': devicelist, 'solution': responselist})
solutions_df = spark.createDataFrame(solutions_df)

solved_df = df.join(solutions_df, df.query_column == solutions_df.error_message, how='left').select(['tower_id', 'device_id', 'device', 'solution']).withColumn('event_timestamp', current_timestamp())

solved_df.write.mode('append').insertInto(f"{CATALOG}.{SCHEMA}.triaged_devices")